# 2022 Sydney University Honours project - Novel kernels for deep learning on light field images

In [6]:
import numpy as np
import os
import PIL
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import sys, glob, os, random
import pandas as pd
print(tf.__version__)

2.9.1


### Data loading

In [17]:
exclude_ref = True
dn = "MPI-LFA"
af = "x8" ###-
img_format = "png"
s = None

Stack 2D images to get a single LFI

In [18]:
def select_sai_range():
    n_sai = 101
    target_n_sai = 49
    mid = n_sai//2
    left = mid - target_n_sai//2
    right = mid + target_n_sai//2
    return left, right

def proc_sai(p):
    # print("proc_sai")
    img = Image.open(p)
    w, h = img.size
    # left, top, right, bottom
    w_offset = int((w-434)/2)
    h_offset = int((h-434)/2)
    l,r = w_offset, w_offset+434
    tp,b = h_offset, h_offset+434
    window = (l,tp,r,b)
    new_img = img.crop(window)
    # print(new_img)
    return np.asarray(new_img)

def flatten_dataset(save_dir, read_dir):
    read_dir = read_dir
    read_img_paths = glob.glob(read_dir+"**/*."+img_format, recursive=True)
    read_img_paths = sorted(read_img_paths)

    save_dir = './'
    save_ref = save_dir+'ref/'
    save_dist = save_dir+'dist/'
    if not os.path.exists(save_ref):
        os.makedirs(save_ref)

    if not os.path.exists(save_dist):
        os.makedirs(save_dist)
        ref_word = 'ALL_REF'


    # print(read_img_paths)
    print(len(read_img_paths)//101)
    left, right  = select_sai_range()

    print(left, right)

    i = 0
    j = 0
    while i < len(read_img_paths):
        # print(i, j)
        if not (left <= j <= right):
          i += 1
          j += 1
          continue

        if j == left:
            to_shape=(7,434,7,434,3)
            lfi = np.zeros(to_shape, dtype=np.uint8)
            frames = []

        p = read_img_paths[i]
        frames.append(p)

        sai = proc_sai(p)
        # print(sai)
        u, v = (j-left)//7, (j-left)%7
        lfi[u,:,v,:,:] = sai

        i += 1
        j += 1
        if j == 75:
            #print(frames)
            j = 0
            i += 26
            parts = p.split(s)
            name = parts[-3]+'-'+parts[-2]
            lfi = lfi.reshape((7*434, 7*434, 3), order='F')

            new_img = Image.fromarray(lfi)

            if ref_word in name:
                new_img.save(save_ref+name+".png",img_format)
            else:
                new_img.save(save_dist+name+".png",img_format)

            print(f"{name} saved.")

In [19]:
data_path = '../../datasets'

#img = Image.open(data_path + '/hci_dataset/training/boxes/input_Cam000.png')
#img.show()

flatten_dataset(data_path + '/hci_dataset/training/boxes/stacked', 
                data_path + '/hci_dataset/training/boxes/')

0
26 74


IndexError: list index out of range

In [20]:
image = tf.keras.utils.load_img(
    data_path + '/hci_dataset/training/boxes/input_Cam000.png',
    grayscale=False,
    color_mode='rgb',
    target_size=None,
    interpolation='nearest',
    keep_aspect_ratio=False
)

input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr]) 
print(input_arr.shape)

(1, 512, 512, 3)
